In [1]:
import numpy as np

In [2]:
dim = 28*28 # dim of the original img
q_dim = 2 # dim of the quantized img

# test data matrix
X = np.asarray([[1,0,0],[1,1,0]])
# test f
f = np.asarray([1, 2, 3, 4, 5, 6])

print(X)
print(f)

[[1 0 0]
 [1 1 0]]
[1 2 3 4 5 6]


In [3]:
# maps a pixel to quantized index
def q_map(x, threshold=0.4):
    if x < threshold:
        return 0
    else:
        return 1
    
# given a data matrix, returns the mask
def q_mask(X):
    mask = np.zeros((X.shape[0], X.shape[1]*q_dim), dtype=np.int)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            mask[i, j*q_dim+q_map(X[i,j])] = 1
    norm_p = np.sum(mask, axis=0) / X.shape[0]
    return mask, norm_p
            
mask, norm_p = q_mask(X)
print(mask)
print(norm_p)

[[0 1 1 0 1 0]
 [0 1 0 1 1 0]]
[0.  1.  0.5 0.5 1.  0. ]


In [4]:
# Playground for numpy
print(mask)
print(mask[:, 2*1])
b = mask[:, 2*1] == 1
print(mask[:, 2*1] == 1)

f_ext = np.tile(f, (mask.shape[0], 1))
print(f_ext)

f_ext[:, 2].fill(0)
print(f_ext)

print(f_ext*mask)
tmp = b.reshape(mask.shape[0], -1)*f_ext*mask
print(tmp)
print(np.sum(np.sum(tmp, axis=1)) / np.sum(b))

a = np.arange(6).reshape(2,3)
print(a)
print(a**2)

[[0 1 1 0 1 0]
 [0 1 0 1 1 0]]
[1 0]
[ True False]
[[1 2 3 4 5 6]
 [1 2 3 4 5 6]]
[[1 2 0 4 5 6]
 [1 2 0 4 5 6]]
[[0 2 0 0 5 0]
 [0 2 0 4 5 0]]
[[0 2 0 0 5 0]
 [0 0 0 0 0 0]]
7.0
[[0 1 2]
 [3 4 5]]
[[ 0  1  4]
 [ 9 16 25]]


In [6]:
# X: data matrix
# feature extracting func
def ACE_step(X, f, mask, norm_p):
    # E[sum f(X_j)|X_i], res is f.dim = data.dim*q_dim
    def expect_cond(X, f):
        res = np.zeros_like(f)
        for j in range(f.shape[0]):
            b = (mask[:, j] == 1)
            f_ext = np.tile(f, (X.shape[0], 1))
            f_ext[:, j].fill(0)
            tmp = b.reshape(mask.shape[0], -1)*f_ext*mask
            if np.sum(b) > 0:
                res[j] = np.sum(np.sum(tmp, axis=1)) / np.sum(b)
        return res
    
    # sqrt(E[sum f(X_i)^2])
    def expect_var(X, f):
        # sum_cov = np.sum(np.sum((np.tile(f, (X.shape[0], 1))*mask)**2))
        # e_cov = sum_cov / X.shape[0]
        # res = np.sqrt(e_cov)
        
        res = np.sqrt(np.sum((f**2) * norm_p))
        return res
    
    e_cond = expect_cond(X, f)
    new_f = f + e_cond
    cov = expect_var(X, new_f)
    
    new_f = new_f / cov
    return new_f

print(X)
print(f)
print(ACE_step(X, f, mask, norm_p))

[[1 0 0]
 [1 1 0]]
[1 2 3 4 5 6]
[0.05675044 0.56750435 0.56750435 0.62425479 0.56750435 0.34050261]


In [7]:
def init_f(dim=28*28, q_dim=2):
    res_dim = dim * q_dim
    res = np.random.rand(res_dim)
    for j in range(dim):
        res[j*q_dim : (j+1)*q_dim] -= np.sum(res[j*q_dim : (j+1)*q_dim])/2
    return res

print(init_f(3))
print(init_f(3))
print(init_f(3))

[-0.13288444  0.13288444 -0.37864779  0.37864779 -0.03680652  0.03680652]
[-0.0964853   0.0964853   0.2896539  -0.2896539  -0.02676792  0.02676792]
[-0.00364475  0.00364475 -0.0639841   0.0639841  -0.04220496  0.04220496]


In [18]:
def ACE_f(X, mask, norm_p, dim=28*28, q_dim=2, epsilon=0.01, max_step=500):
    f_1 = init_f(dim, q_dim)
    for _ in range(max_step):
        f_2 = ACE_step(X, f_1, mask, norm_p)
        delta = np.sum((f_1 - f_2)**2)
        f_1 = f_2
        # print(delta)
        if delta < epsilon:
            break
    return f_1

l = ACE_f(X, mask, norm_p, 3)
print(l)

[ 0.03845616 -0.57735018 -0.57680789 -0.57789248 -0.57735018 -0.01707282]


In [19]:
def norm_dot(f1, f2, norm_p):
    res = np.sum(f1 * f2 * norm_p)
    return res

In [20]:
def ACE_fs(X, k, mask, norm_p, dim=28*28, q_dim=2, epsilon=0.01, max_step=500):
    res = []
    for i in range(k):
        f_i = ACE_f(X, mask, norm_p, dim, q_dim, epsilon, max_step)
        f_tmp = np.zeros_like(f_i)
        for f_m in res:
            f_tmp = f_tmp + norm_dot(f_i, f_m, norm_p)*f_m
        f_i -= f_tmp
        res.append(f_i)
    return res

fs = ACE_fs(X, 3, mask, norm_p, 3)
print(fs)
print(np.sum(norm_dot(fs[0], fs[1], norm_p)))
print(np.sum(norm_dot(fs[0], fs[2], norm_p)))
print(np.sum(norm_dot(fs[1], fs[2], norm_p)))

[array([-0.04201404,  0.57732526,  0.58663271,  0.56801781,  0.57732526,
       -0.02213321]), array([-4.14386757e-02,  2.75925530e-05, -5.10696558e-03,  5.16215069e-03,
        2.75925530e-05, -1.55806921e-02]), array([-2.91650878e-02, -8.63773463e-05,  1.59871446e-02, -1.61598993e-02,
       -8.63773463e-05, -1.35957280e-02])]
-1.6295558652457132e-16
-1.4837984356821932e-16
-8.253758288939527e-05


In [24]:
a = np.arange(40).reshape(8, 5)
print(a)

idxs = np.random.choice(a.shape[0], 3, False)
print(idxs)
print(a[idxs, :])

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]
 [25 26 27 28 29]
 [30 31 32 33 34]
 [35 36 37 38 39]]
[6 2 5]
[[30 31 32 33 34]
 [10 11 12 13 14]
 [25 26 27 28 29]]
